<a href="https://colab.research.google.com/github/saikrishna1108/5106_Final_Project/blob/main/Transformer_Classification_with_10_margin_seq_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
os.chdir('/content/drive/MyDrive/rtml/5106_Final_Project/5106_Final_Project')

In [1]:
import os
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
import math

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5, augment=True):
        self.data = []
        self.sequence_length = sequence_length
        self.num_classes = (105 - 30) // 10 + 1  # Speed classes from 30-39, ..., 100-105
        self.augment = augment  # Enable or disable augmentation
        self.preprocess_data(directory)
        self.normalize_features()

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                speed_class = 0 if speed < 30 else (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        points = line.strip().split(',')
                        if len(points) == 6:
                            frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if len(points) == 7:
                            frame,class_id, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1, y1, x2, y2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = self.extract_features(frames)
                            overlap = 5  # Overlap for sequence extraction
                            for start_idx in range(0, len(features) - self.sequence_length + 1, self.sequence_length - overlap):
                                end_idx = start_idx + self.sequence_length
                                sequence = features[start_idx:end_idx]
                                self.data.append((sequence, speed_class))

    def extract_features(self, frames):
        features = []
        for i in range(1, len(frames)):
            current_frame = frames[i]
            previous_frame = frames[i-1]
            features.append(self.compute_frame_features(current_frame, previous_frame))
        return features

    def compute_frame_features(self, current_frame, previous_frame):
        x1, y1, x2, y2 = current_frame
        px1, py1, px2, py2 = previous_frame

        width, height = x2 - x1, y2 - y1
        p_width, p_height = px2 - px1, py2 - py1

        x2_change = x2 - px2
        y2_change = y2 - py2
        x1_change = x1 - px1
        y1_change = y1 - py1

        width_change = width - p_width
        height_change = height - p_height
        area_change = (width * height) - (p_width * p_height)
        perimeter_change = (2 * (width + height)) - (2 * (p_width + p_height))

        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        p_center_x, p_center_y = (px1 + px2) / 2, (py1 + py2) / 2
        center_x_change = center_x - p_center_x
        center_y_change = center_y - p_center_y
        distance_moved = math.sqrt(center_x_change ** 2 + center_y_change ** 2)

        velocity = distance_moved/0.03  # Assuming constant frame rate
        p_velocity = (math.sqrt((px2 - px1) ** 2 + (py2 - py1) ** 2))/0.03
        acceleration = abs(velocity - p_velocity)

        feature_vector = [x1_change, y1_change, x2_change, y2_change,
                          center_x_change, center_y_change, distance_moved, velocity, acceleration]
        return feature_vector

    def normalize_features(self):
        all_features = [feature for sequence, _ in self.data for feature in sequence]
        all_features = np.array(all_features)
        self.mean = np.mean(all_features, axis=0)
        self.std = np.std(all_features, axis=0)
        for i, (sequence, speed_class) in enumerate(self.data):
            normalized_sequence = (sequence - self.mean) / (self.std)
            self.data[i] = (normalized_sequence, speed_class)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        if self.augment:
            inputs = self.apply_augmentation(inputs)
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)

    def apply_augmentation(self, inputs):
        if random.random() > 0.5:
            inputs = self.add_noise(inputs)
        if random.random() > 0.5:
            inputs = self.time_shift(inputs, shift=random.choice([-1, 1]))
        if random.random() > 0.5:
            inputs = self.scale_features(inputs, scale=random.uniform(0.9, 1.1))
        if random.random() > 0.5:
            inputs = self.mirror_features(inputs)
        return inputs

    def add_noise(self, features, noise_level=0.05):
        noise = np.random.normal(0, noise_level, features.shape)
        return features + noise

    def time_shift(self, features, shift=1):
        if shift > 0:
            return np.vstack([np.zeros((shift, features.shape[1])), features[:-shift]])
        elif shift < 0:
            return np.vstack([features[-shift:], np.zeros((-shift, features.shape[1]))])
        return features

    def scale_features(self, features, scale=1.1):
        return features * scale

    def mirror_features(self, features):
        features_copy = features.copy()
        features_copy[:, [0, 2]] = -features_copy[:, [0, 2]]  # Assume these indices are the x-coordinates
        return features_copy


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.embedding = nn.Conv1d(in_channels=feature_size, out_channels=hidden_dim, kernel_size=1)
        self.pos_encoder = PositionalEncoding(hidden_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=8, dim_feedforward=hidden_dim * 4, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=10)
        self.fc1 = nn.Linear(hidden_dim, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = x.permute(1, 0, 2)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.permute(1, 0, 2)  # Switch back to (batch, seq_len, features)
        x = x[:, -1, :]  # Only use the last sequence output for prediction
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.bias, 0)


In [10]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


Mounted at /content/drive


In [15]:

def train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs):
    for epoch in range(epochs):
        model.train()
        Train_total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # First, update the parameters with the current learning rate

            Train_total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)

        scheduler.step()  # After optimizer updates, adjust the learning rate

        if epoch % 5 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            total_loss = 0
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)

            train_accuracy = 100 * correct_train / total_train
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {Train_total_loss / len(train_loader)} Test Loss: {total_loss / len(train_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {test_accuracy:.2f}%')


In [8]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ExponentialLR

# Assuming your SpeedPredictor and SpeedDataset are already defined and imported

# Initialize datasets
train_dataset = SpeedDataset('./Dataset/Train/', sequence_length=15)
test_dataset = SpeedDataset('./Dataset/Test/', sequence_length=15)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

len(train_dataset)


3823

In [11]:
def extract_labels(data_loader):
    labels = []
    for _, y in data_loader:
        labels.append(y)
    return torch.cat(labels)  # Concatenate list of tensors into a single tensor

# Extract labels from the training DataLoader
train_labels = extract_labels(train_loader)

# Compute class weights using sklearn's compute_class_weight
from sklearn.utils.class_weight import compute_class_weight

# Convert train_labels to a numpy array and calculate class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels.numpy()),
    y=train_labels.numpy()
)

# Convert class weights to a tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float, device=device)


In [16]:
# Model setup
max_classes = train_dataset.num_classes
model = SpeedPredictor(sequence_length=15, feature_size=9, hidden_dim=128, output_size=max_classes)
model.to(device)  # Ensure model is on the appropriate device
model.apply(init_weights)
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor).to(device)  # Ensure loss function is on the appropriate device
optimizer = Adam(model.parameters(), lr=0.0001)
scheduler = ExponentialLR(optimizer, gamma=0.99999999)  # Learning rate scheduler

# Train the model
train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=200)


Epoch 1: Train Loss: 2.6054155429204306 Test Loss: 0.1750391085942586, Train Accuracy: 12.76%, Validation Accuracy: 18.83%
Epoch 6: Train Loss: 1.7581575632095336 Test Loss: 0.14619530042012532, Train Accuracy: 29.11%, Validation Accuracy: 23.70%
Epoch 11: Train Loss: 1.5252215246359506 Test Loss: 0.10261762539545695, Train Accuracy: 37.14%, Validation Accuracy: 48.70%
Epoch 16: Train Loss: 1.429120800892512 Test Loss: 0.11582405765851339, Train Accuracy: 40.62%, Validation Accuracy: 42.86%
Epoch 21: Train Loss: 1.3457391142845154 Test Loss: 0.09660096168518066, Train Accuracy: 43.71%, Validation Accuracy: 48.70%
Epoch 26: Train Loss: 1.3384902477264404 Test Loss: 0.09619352420171103, Train Accuracy: 43.26%, Validation Accuracy: 49.35%
Epoch 31: Train Loss: 1.32274695734183 Test Loss: 0.09318316479523976, Train Accuracy: 43.26%, Validation Accuracy: 48.38%
Epoch 36: Train Loss: 1.2897688746452332 Test Loss: 0.08947171171506246, Train Accuracy: 45.30%, Validation Accuracy: 51.95%
Epoch 

In [17]:
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)

Confusion Matrix and Classification Report:
[[41  3  2  1  0  0  0  0]
 [10 31 14  0  4  1  1  0]
 [ 1 11 24  5  3  0  2  0]
 [ 2  1  7 24  8  6  5  6]
 [ 0  1  2 10 14  6  1  0]
 [ 0  0  0  2  5 17  8  7]
 [ 0  0  0  0  0  0  0  6]
 [ 0  0  0  0  0  0  0 16]]
               precision    recall  f1-score   support

  Class 30-39       0.76      0.87      0.81        47
  Class 40-49       0.66      0.51      0.57        61
  Class 50-59       0.49      0.52      0.51        46
  Class 60-69       0.57      0.41      0.48        59
  Class 70-79       0.41      0.41      0.41        34
  Class 80-89       0.57      0.44      0.49        39
  Class 90-99       0.00      0.00      0.00         6
Class 100-109       0.46      1.00      0.63        16

     accuracy                           0.54       308
    macro avg       0.49      0.52      0.49       308
 weighted avg       0.57      0.54      0.54       308



54.22077922077922